##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

!printenv

In [1]:
!pip install -U azureml-core
!pip install -U azureml-telemetry
!pip install -U azureml-tensorboard
!pip install -U tensorboard-plugin-
!pip install -U azureml-train

/bin/bash: pip: command not found
/bin/bash: pip: command not found
/bin/bash: pip: command not found
/bin/bash: pip: command not found
/bin/bash: pip: command not found


# Get started with TensorBoard

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tensorboard/get_started"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/get_started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/get_started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/get_started.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In machine learning, to improve something you often need to be able to measure it. TensorBoard is a tool for providing the measurements and visualizations needed during the machine learning workflow. It enables tracking experiment metrics like loss and accuracy, visualizing the model graph, projecting embeddings to a lower dimensional space, and much more.

This quickstart will show how to quickly get started with TensorBoard. The remaining guides in this website provide more details on specific capabilities, many of which are not included here. 

In [15]:
# Load the TensorBoard notebook extension
#%load_ext tensorboard
!pip install azureml-tensorboard

/bin/bash: pip: command not found


In [1]:
from azureml.core import Experiment, Workspace, Run
import tensorflow as tf
import datetime
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

experiment = Experiment(workspace=ws, name='tensorflow_aml')
# start logging for the run
run = experiment.start_logging()

# access the run id for use later
run_id = run.id


Workspace name: mlw-dev-geog-01
Azure region: eastus
Subscription id: 062bbb35-45d7-40c6-937f-a43ab3667b0f
Resource group: rg-dev-geog-eastus


In [2]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

Using the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset as the example, normalize the data and write a function that creates a simple Keras model for classifying the images into 10 classes.

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

## Using TensorBoard with Keras Model.fit()

When training with Keras's [Model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit), adding the `tf.keras.callbacks.TensorBoard` callback ensures that logs are created and stored. Additionally, enable histogram computation every epoch with `histogram_freq=1` (this is off by default)

Place the logs in a timestamped subdirectory to allow easy selection of different training runs.

In [4]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# use the callbacks to endsure that the logs are created and stored
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2170 - accuracy: 0.9363 - val_loss: 0.1135 - val_accuracy: 0.9657
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0962 - accuracy: 0.9703 - val_loss: 0.0783 - val_accuracy: 0.9754
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0685 - accuracy: 0.9788 - val_loss: 0.0782 - val_accuracy: 0.9767
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0527 - accuracy: 0.9832 - val_loss: 0.0722 - val_accuracy: 0.9773
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0439 - accuracy: 0.9855 - val_loss: 0.0672 - val_accuracy: 0.9798


Start TensorBoard through the command line or within a notebook experience. The two interfaces are generally the same. In notebooks, use the `%tensorboard` line magic. On the command line, run the same command without "%".

/bin/bash: tensorboard: command not found


<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_model_fit.png?raw=1"/> -->

A brief overview of the dashboards shown (tabs in top navigation bar):

* The **Scalars** dashboard shows how the loss and metrics change with every epoch. You can use it to also track training speed, learning rate, and other scalar values.
* The **Graphs** dashboard helps you visualize your model. In this case, the Keras graph of layers is shown which can help you ensure it is built correctly. 
* The **Distributions** and **Histograms** dashboards show the distribution of a Tensor over time. This can be useful to visualize weights and biases and verify that they are changing in an expected way.

Additional TensorBoard plugins are automatically enabled when you log other types of data. For example, the Keras TensorBoard callback lets you log images and embeddings as well. You can see what other plugins are available in TensorBoard by clicking on the "inactive" dropdown towards the top right.

## Using TensorBoard with other methods


When training with methods such as [`tf.GradientTape()`](https://www.tensorflow.org/api_docs/python/tf/GradientTape), use `tf.summary` to log the required information.

Use the same dataset as above, but convert it to `tf.data.Dataset` to take advantage of batching capabilities:

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

The training code follows the [advanced quickstart](https://www.tensorflow.org/tutorials/quickstart/advanced) tutorial, but shows how to log metrics to TensorBoard. Choose loss and optimizer:

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

Create stateful metrics that can be used to accumulate values during training and logged at any point:

In [7]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')



Define the training and test functions:

In [8]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

Set up summary writers to write the summaries to disk in a different logs directory:

In [9]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

run.log("current_time", current_time)
run.log("train_log_dir", train_log_dir)
run.log("test_log_dir", test_log_dir)
run.log("train_summary_writer", train_summary_writer)
run.log("test_summary_writer", test_summary_writer)



Start training. Use `tf.summary.scalar()` to log metrics (loss and accuracy) during training/testing within the scope of the summary writers to write the summaries to disk. You have control over which metrics to log and how often to do it. Other `tf.summary` functions enable logging other types of data.

In [10]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))
  run.log("epoch", epoch+1)
  run.log("loss", train_loss.result())
  run.log("accuracy%", train_accuracy.result()*100)
  run.log("test loss", test_loss.result())
  run.log("test_accuracy", test_accuracy.result())



  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.2403009533882141, Accuracy: 93.04166412353516, Test Loss: 0.11897052824497223, Test Accuracy: 96.5
Epoch 2, Loss: 0.10426203906536102, Accuracy: 96.8550033569336, Test Loss: 0.0883815586566925, Test Accuracy: 97.37999725341797
Epoch 3, Loss: 0.07215358316898346, Accuracy: 97.77833557128906, Test Loss: 0.07186562567949295, Test Accuracy: 97.65999603271484
Epoch 4, Loss: 0.05428000167012215, Accuracy: 98.30667114257812, Test Loss: 0.0707816556096077, Test Accuracy: 97.6199951171875
Epoch 5, Loss: 0.04376479610800743, Accuracy: 98.5816650390625, Test Loss: 0.05892794206738472, Test Accuracy: 98.05999755859375


Open TensorBoard again, this time pointing it at the new log directory. We could have also started TensorBoard to monitor training while it progresses.

In [11]:
run.complete()

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_gradient_tape.png?raw=1"/> -->

That's it! You have now seen how to use TensorBoard both through the Keras callback and through `tf.summary` for more custom scenarios. 

## TensorBoard.dev: Host and share your ML experiment results

[TensorBoard.dev](https://tensorboard.dev) is a free public service that enables you to upload your TensorBoard logs and get a permalink that can be shared with everyone in academic papers, blog posts, social media, etc.  This can enable better reproducibility and collaboration.

To use TensorBoard.dev, run the following command:


!tensorboard dev upload \
  --logdir logs/fit \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot

Note that this invocation uses the exclamation prefix (`!`) to invoke the shell
rather than the percent prefix (`%`) to invoke the colab magic.  When invoking this command from the command line there is no need for either prefix.

View an example [here](https://tensorboard.dev/experiment/EDZb7XgKSBKo6Gznh3i8hg/#scalars).

For more details on how to use TensorBoard.dev, see https://tensorboard.dev/#get-started